In [1]:
# Dependencies and Setup
import pandas as pd
import csv
import numpy as np
from geopy.geocoders import Nominatim
from sqlalchemy import create_engine
from databaseconfig import password

# EXTRACT

#### Real DATASETS

In [2]:
# File to Load 
crime_to_load = "Resources/Baltimore_2019_Crime_Only.csv"
cctv_to_load = "Resources/Baltimore_CCTV_Locations.csv"

#### Test for API calls

In [ ]:
# File to Load 
#crime_to_load = "Resources/Baltimore_2019_Crime_Only-head.csv"
#cctv_to_load = "Resources/Baltimore_CCTV_Locations-head.csv"

In [3]:
# Read the CSV files
crime_df = pd.read_csv(crime_to_load)
cctv_df = pd.read_csv(cctv_to_load)

# TRANSFORM

In [4]:
# Changing "Location 1" form cctv_df to 'latitude' and 'longitiude'
# This is required for geocoding which we want to use to find addresses of deployed CCTV

# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cctv_df['Location 1']:
    # Try to,
    try:
        #Remove ( from the string
        row = row.replace("(", "")
        row = row.replace(")", "")
        # Split the row by comma, convert to float, and append
        # everything before the comma to lat
        lat.append(float(row.split(',')[0]))
        # Split the row by comma, convert to float, and append
        # everything after the comma to lon
        lon.append(float(row.split(',')[1]))
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
cctv_df['latitude'] = lat
cctv_df['longitude'] = lon

### Test geolocator and pre-function tests

In [ ]:
# Test how does geolocator.reverse works
#geolocator = Nominatim(user_agent="Baltimore_Crime")
#location = geolocator.reverse('39.290996, -76.621074')

In [ ]:
#location.address

In [ ]:
#lon_lat_list = list(zip(cctv_df['latitude'], cctv_df['longitude']))
#len(lon_lat_list)

In [ ]:
#str(lon_lat_list[0]).replace('(', '').replace(')', '')

#### Translate latitudes and longitudes into neighbourhood

In [5]:
def convert_to_address(row) -> str:
#     print(row)
    lat = row['latitude']
    lon = row['longitude']
    input_lon_lat = str(lat) + ',' + str(lon)
    geolocator = Nominatim(user_agent="Baltimore_Crime")
    location = geolocator.reverse(input_lon_lat)
    print(location.address)
    return location.address

#### Funciton troubleshooting

In [ ]:
#sample = lon_lat_list[0]
#sample

In [6]:
cctv_df['address'] = cctv_df.apply(lambda row: convert_to_address(row), axis = 1)

King Of Lexington Jewelry, 110, North Eutaw Street, Seton Hill, Baltimore, Maryland, 21201, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
Medusa, 401, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
400, West Redwood Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
The Goddess, West Lombard Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
South Eutaw Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
511, West Pratt Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
601, West Lexington Street, Seton Hill, Baltimore, Maryland, 21201, USA
500, West Faye

Eastern Avenue, Fells Point, Baltimore, Maryland, 21231, USA
423, South Ann Street, Fells Prospect, Baltimore, Maryland, 21231, USA
South Broadway & Fleet Street, South Broadway, Fells Point, Baltimore, Maryland, 21231, USA
1704, Fleet Street, Fells Point, Baltimore, Maryland, 21231, USA
536, South Ann Street, Fells Prospect, Baltimore, Maryland, 21231, USA
1442, Fleet Street, Harbor East, Baltimore, Maryland, 21231, USA
600, South Eden Street, Harbor East, Baltimore, Maryland, 21205, USA
1703, Aliceanna Street, Fells Point, Baltimore, Maryland, 21231, USA
1615, Aliceanna Street, Fells Point, Baltimore, Maryland, 21231, USA
The Admiral's Cup, 1647, Thames Street, Fells Point, Baltimore, Maryland, 21231, USA
2776, Berea Road, Cherry Hill, Baltimore, Maryland, 21225, USA
1210, Round Road, Cherry Hill, Baltimore, Maryland, 21225, USA
1399, Slater Road, Cherry Hill, Baltimore, Maryland, 21225, USA
2705, Berea Road, Cherry Hill, Baltimore, Maryland, 21225, USA
1211, Shellbanks Road, Cherry 

2510, McElderry Street, McElderry Park, Baltimore, Maryland, 21205, USA
2541, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
2620, East Madison Street, McElderry Park, Baltimore, Maryland, 21205, USA
2642, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
601, North Belnord Avenue, McElderry Park, Baltimore, Maryland, 21205, USA
2739, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
800, North Streeper Street, McElderry Park, Baltimore, Maryland, 21205, USA
2901, McElderry Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, USA
700, North Curley Street, Baltimore - Linwood, Baltimore, Maryland, 21205, USA
3000, McElderry Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, USA
3021, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
3027, East Madison Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, USA
3118, East Monument Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, 

2752, Ellicott Driveway, Winchester, Baltimore, Maryland, 21216, USA
1027, Poplar Grove Street, Winchester, Baltimore, Maryland, 21216, USA
2800, Mosher Street, Mosher, Baltimore, Maryland, 21216, USA
Alexander Hamilton Elementary School, West Lafayette Avenue, Franklintown Road, Baltimore, Maryland, 21216, USA
719, North Longwood Street, Franklintown Road, Baltimore, Maryland, 21216, USA
714, Poplar Grove Street, Winchester, Baltimore, Maryland, 21216, USA
2838, Edmondson Avenue, Mosher, Baltimore, Maryland, 21223, USA
2594, Edmondson Avenue, Mosher, Baltimore, Maryland, 21223, USA
King Of Lexington Jewelry, 110, North Eutaw Street, Seton Hill, Baltimore, Maryland, 21201, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
Medusa, 401, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
400, West Redwood Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
The Goddess, West Lombard Street, Ridgleys De

100, South Washington Street, Fells Prospect, Baltimore, Maryland, 21231, USA
128, South Broadway, Washington Hill, Baltimore, Maryland, 21231, USA
201, East Pratt Street, Perkins Homes, Baltimore, Maryland, 21223, USA
Gough Street, Perkins Homes, Baltimore, Maryland, 21231, USA
255, South Regester Street, Washington Hill, Baltimore, Maryland, 21231, USA
330, South Broadway, Perkins Homes, Baltimore, Maryland, 21231, USA
355, Bank Street, Perkins Homes, Baltimore, Maryland, 21231, USA
400, South Caroline Street, Perkins Homes, Baltimore, Maryland, 21231, USA
Eastern Avenue, Fells Point, Baltimore, Maryland, 21231, USA
423, South Ann Street, Fells Prospect, Baltimore, Maryland, 21231, USA
South Broadway & Fleet Street, South Broadway, Fells Point, Baltimore, Maryland, 21231, USA
1704, Fleet Street, Fells Point, Baltimore, Maryland, 21231, USA
536, South Ann Street, Fells Prospect, Baltimore, Maryland, 21231, USA
1442, Fleet Street, Harbor East, Baltimore, Maryland, 21231, USA
600, South

629, North Collington Avenue, Butchers Hill, Baltimore, Maryland, 21205, USA
722, North Madeira Street, Butchers Hill, Baltimore, Maryland, 21205, USA
2301, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
2321, McElderry Street, McElderry Park, Baltimore, Maryland, 21205, USA
701, North Montford Avenue, McElderry Park, Baltimore, Maryland, 21205, USA
801, North Port Street, McElderry Park, Baltimore, Maryland, 21205, USA
2442, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
2510, McElderry Street, McElderry Park, Baltimore, Maryland, 21205, USA
2541, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
2620, East Madison Street, McElderry Park, Baltimore, Maryland, 21205, USA
2642, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
601, North Belnord Avenue, McElderry Park, Baltimore, Maryland, 21205, USA
2739, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
800, North Streeper Street,

1649, Poplar Grove Street, Winchester, Baltimore, Maryland, 21216, USA
2801, Baker Street, Gilmor Homes, Baltimore, Maryland, 21216, USA
1412, Bloomingdale Road, Winchester, Baltimore, Maryland, 21216, USA
2900, Presstman Street, Winchester, Baltimore, Maryland, 21216, USA
2906, Belmont Avenue, Winchester, Baltimore, Maryland, 21216, USA
1405, North Rosedale Street, Rosemont, Baltimore, Maryland, 21216, USA
1158, North Longwood Street, Franklintown Road, Baltimore, Maryland, 21216, USA
2900, Ellicott Driveway, Winchester, Baltimore, Maryland, 21216, USA
2752, Ellicott Driveway, Winchester, Baltimore, Maryland, 21216, USA
1027, Poplar Grove Street, Winchester, Baltimore, Maryland, 21216, USA
2800, Mosher Street, Mosher, Baltimore, Maryland, 21216, USA
Alexander Hamilton Elementary School, West Lafayette Avenue, Franklintown Road, Baltimore, Maryland, 21216, USA
719, North Longwood Street, Franklintown Road, Baltimore, Maryland, 21216, USA
714, Poplar Grove Street, Winchester, Baltimore,

Pier 6 Concert Pavilion, 731, Eastern Avenue, Little Italy, Baltimore, Maryland, 21202, USA
East Conway Street, Inner Harbor, Baltimore, Maryland, 21202, USA
West Conway Street, Otterbein, Baltimore, Maryland, 21202:21230, USA
176, West Hamburg Street, Sharp-Leadenhall, Baltimore, Maryland, 21230, USA
Baltimore Convention Center, 1, West Pratt Street, Inner Harbor, Baltimore, Maryland, 21201, USA
One East Pratt Street, 1, East Pratt Street, Inner Harbor, Baltimore, Maryland, 21202, USA
Northeast Market, 2101, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
Northeast Market, 2101, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
Northeast Market, 2101, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
Northeast Market, 2101, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
Parking Authority of Baltimore City, McElderry Street, McElderry Park, Baltimore, Maryland, 21205, USA
1631 H, Pennsylvania Avenue, U

299, Gough Street, Perkins Homes, Baltimore, Maryland, 21231, USA
354, South Spring Court, Perkins Homes, Baltimore, Maryland, 21231, USA
331, South Spring Court, Perkins Homes, Baltimore, Maryland, 21231, USA
1413, Gough Street, Perkins Homes, Baltimore, Maryland, 21231, USA
319, South Ballou Court, Perkins Homes, Baltimore, Maryland, 21231, USA
330, South Ballou Court, Perkins Homes, Baltimore, Maryland, 21231, USA
326, South Dallas Court, Perkins Homes, Baltimore, Maryland, 21231, USA
338, Mason Court, Perkins Homes, Baltimore, Maryland, 21231, USA
329, South Dallas Court, Perkins Homes, Baltimore, Maryland, 21231, USA
326, South Dallas Court, Perkins Homes, Baltimore, Maryland, 21231, USA
231, East Street, Old Town, Baltimore, Maryland, 21202, USA
382, Forrest Street, Old Town, Baltimore, Maryland, 21202, USA
1198, New Hope Circle, Jonestown, Baltimore, Maryland, 21202, USA
1021, New Hope Circle, Jonestown, Baltimore, Maryland, 21202, USA
201, East Street, Old Town, Baltimore, Mary

In [7]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,address
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,"King Of Lexington Jewelry, 110, North Eutaw St..."
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,"University of Maryland Baltimore, West Baltimo..."
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,"Medusa, 401, West Baltimore Street, Ridgleys D..."
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,"400, West Redwood Street, Ridgleys Delight, Ba..."
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,"The Goddess, West Lombard Street, Ridgleys Del..."


### Divide full entry from geolocator into separate columns

In [26]:
cctv_df[['Building','No.', 'Street', 'Neighborhood', 'City', 'State', 'Country', 'Other']] = cctv_df.address.str.split(',',expand=True,)

In [27]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,address,Building,No.,Street,Neighborhood,City,State,Country,Other,Other2
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,"King Of Lexington Jewelry, 110, North Eutaw St...",King Of Lexington Jewelry,110,110,North Eutaw Street,Seton Hill,Baltimore,Maryland,21201,USA
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,"University of Maryland Baltimore, West Baltimo...",University of Maryland Baltimore,West Baltimore Street,West Baltimore Street,Ridgleys Delight,Baltimore,Maryland,21223,USA,None
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,"Medusa, 401, West Baltimore Street, Ridgleys D...",Medusa,401,401,West Baltimore Street,Ridgleys Delight,Baltimore,Maryland,21201,USA
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,"400, West Redwood Street, Ridgleys Delight, Ba...",400,West Redwood Street,West Redwood Street,Ridgleys Delight,Baltimore,Maryland,21201,USA,None
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,"The Goddess, West Lombard Street, Ridgleys Del...",The Goddess,West Lombard Street,West Lombard Street,Ridgleys Delight,Baltimore,Maryland,21201,USA,None


#### Store DF into DB before we run out of API requests

In [20]:
# Open connection to Postres
db_conn = "postgresql://postgres:" + password + "@localhost:5432/baltimore_crime_db"
engine = create_engine(db_conn)
conn = engine.connect()

In [19]:
cctv_df.to_sql('cctv_full', engine)

#### Store backup of the cctv_full table

In [21]:
cctv_df.to_sql('cctv_full_bck', engine)

### Clean data in cctv_df

In [23]:
cctv_df.drop(columns=['cameraLocation', 'cameraProject', 'Location 1', 'address', 'Building', 'No.', 'Street', 'State', 'Country', 'Other'])

,cameraNumber,latitude,longitude,Neighborhood,City
0,1,39.290996,-76.621074,Seton Hill,Baltimore
1,2,39.290488,-76.623665,Baltimore,Maryland
2,3,39.289324,-76.620985,Ridgleys Delight,Baltimore
3,4,39.288678,-76.620948,Baltimore,Maryland
4,5,39.287528,-76.620854,Baltimore,Maryland
5,6,39.285245,-76.620675,Maryland,21201
6,7,39.286257,-76.622129,Baltimore,Maryland
7,8,39.287415,-76.623616,Baltimore,Maryland
8,9,39.289222,-76.623695,Baltimore,Maryland
9,10,39.290333,-76.623777,Baltimore,Maryland


In [24]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,address,Building,No.,Street,Neighborhood,City,State,Country,Other
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,"King Of Lexington Jewelry, 110, North Eutaw St...",King Of Lexington Jewelry,110,North Eutaw Street,Seton Hill,Baltimore,Maryland,21201,USA
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,"University of Maryland Baltimore, West Baltimo...",University of Maryland Baltimore,West Baltimore Street,Ridgleys Delight,Baltimore,Maryland,21223,USA,None
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,"Medusa, 401, West Baltimore Street, Ridgleys D...",Medusa,401,West Baltimore Street,Ridgleys Delight,Baltimore,Maryland,21201,USA
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,"400, West Redwood Street, Ridgleys Delight, Ba...",400,West Redwood Street,Ridgleys Delight,Baltimore,Maryland,21201,USA,None
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,"The Goddess, West Lombard Street, Ridgleys Del...",The Goddess,West Lombard Street,Ridgleys Delight,Baltimore,Maryland,21201,USA,None


In [ ]:
cctv_df=cctv_df['']